In [1]:
import pandas as pd
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from utils import *

# 解决方案总结

我们发现数据抽样过于稀疏,导致每天给的7天训练数据对应用户来说独立(只有300左右回头客个),训练历史数据对未来基本预测没有影响,所以把每天数据独立出来,变成测试样本,原因如下

- 第一对于每天数据,统计特征独立,可以分离成为user, hotel, room 三个维度
- 当然对应 roomtag 和 roomservice 有异常数据,可以提取出来
- 对应一天内订购两次的我们把它作为异常数据处理(测试集没有)

In [2]:
train_df = pd.read_pickle('../dataset/train.pkl')

In [3]:
test_df = pd.read_pickle('../dataset/test.pkl')

In [144]:
train_df = train_df.set_index('orderdate').loc['2017-06-11'].reset_index()

# 任务描述 

### 根据6月5号到6月11号的数据预测6月12号的数据

根据给出的oderid预测用户在这个酒店预定的子房型,每个子房型会有一个猜你喜欢(roomtag_1)

使用30524个订单预测34632个订单

训练集和预测集的异常数据

- roomtag_5 0
- roomtag_6 0
- orderbehavior_3_ratio_1month 0 过去一个月发生行为3的占比
- orderbehavior_4_ratio_1month 0 过去一个月发生行为4的占比
- orderbehavior_5_ratio_1month 0 过去一个月发生行为5的占比
- orderid_lastord 0 似乎是错误的，时间有，或者全部归0了
- roomtag_6_lastord 0 上次预定的标签为0

## roomid 子房型 basicroomid 物理房型号 hotelid 酒店号 三者关系

每个酒店物理房型固定，就是basicroomid 对应 唯一的hotelid

物理房型下面可能有多个子房型（具体为旅行社不同，价格也不同），但是单个子房型不对应唯一物理房型，也就是子房型可能对应多个物理房型，训练集很奇怪有7105个roomid对应的两个hotelid和basicroomid，测试集只有3765，原以为是异常数据但是测试集占了20%(2万）的数据，删掉容易失真




### 检测三个id之间的关系

In [3]:
xdf = train_df.drop_duplicates(['hotelid', 'basicroomid'])[['hotelid', 'basicroomid']]

In [6]:
(xdf.groupby('basicroomid').count().hotelid != 1).sum()

0

In [7]:
tdf = train_df.drop_duplicates(['roomid', 'hotelid'])[['roomid', 'hotelid']]

In [8]:
ndf = tdf.groupby('roomid').count()

In [9]:
ndf.loc[ndf.hotelid!=1].shape

(7105, 1)

## 订单时间异常数据

用户 -- 102620 80805 72575 52982 22135 18034 过去订单和订单时间不满足时序关系，时序错乱（猜测为异常数据，或者取消了订单）
观察 80805 用户可知，其在同一家酒店9988住了四天，然而仍然时序错乱，推测可能为异常数据，需要清理

用户 31170 2522 正常

训练集总共29724个用户只有一次订单记录，228个两次订单，7个三次以上
测试集总共34632个用户全部只有一次订单记录

7个三次以上在测试集只有三个有记录，两个是在同一酒店预定，其中一个可以确定未来要预定的房间

### 由于异常数据较为少，所以前期不进行清理

In [13]:
cols = train_df.columns.str.extract('(user_)')

ucols = list(train_df.columns[cols.notnull()])+['uid']

udf = train_df[list(train_df.columns[cols.notnull()])+['uid']]

udf.columns

udf.shape

train_df.loc[]

train_df.loc[train_df.uid==17724].drop_duplicates([
    'user_confirmtime', 'user_avgadvanceddate', 'user_avgstar',
    'user_avggoldstar', 'user_avgrecommendlevel', 'user_avgroomnum'
])[ucols]

train_df.orderdate_lastord.value_counts().sort_values()

train_df.loc[train_df.uid==17724]

train_df.loc[train_df.uid==17724][ucols+['orderid', 'orderdate', 'orderdate_lastord']].fillna(0).drop_duplicates([
    'user_confirmtime', 'user_avgadvanceddate', 'user_avgstar',
    'user_avggoldstar', 'user_avgrecommendlevel', 'user_avgroomnum', 'uid', 'orderid'
])

ntdf = train_df.drop_duplicates([
    'user_confirmtime', 'user_avgadvanceddate', 'user_avgstar',
    'user_avggoldstar', 'user_avgrecommendlevel', 'user_avgroomnum', 'uid'
])

uc = ntdf.groupby('uid').count()

tntdf = test_df.drop_duplicates([
    'user_confirmtime', 'user_avgadvanceddate', 'user_avgstar',
    'user_avggoldstar', 'user_avgrecommendlevel', 'user_avgroomnum', 'uid'
])

test_df.drop_duplicates('orderid').shape

tuc = tntdf.groupby('uid').count()

v = train_df.uid.unique()

[x for x in test_df.uid.unique() if x in v]

uc_index = uc.loc[uc.orderid==3].index

[x for x in d if x in uc_index]

train_df.loc[(train_df.uid==18034)&(train_df.orderlabel==1)]

test_df.loc[(test_df.uid==18034)&(test_df.roomid!=11037555)]

tuc.loc[tuc.orderid==1].shape

tuc.loc[tuc.orderid==2].shape

tuc.loc[tuc.orderid==3].shape

train_df.orderdate_lastord.value_counts().sort_index()

train_df.orderid_lastord

def check_time_error(train_df, uid):
    df = train_df.loc[train_df.uid == uid][ucols + [
        'orderid', 'orderdate', 'orderdate_lastord', 'hotelid_lastord'
    ]].fillna(0).drop_duplicates(ucols)
    if np.all(df.sort_values('orderdate_lastord').index == df.sort_values('orderdate').index):
        return True
        print(df.uid.unique()[0])

df = train_df.loc[train_df.uid == 102620][ucols + [
    'orderid', 'orderdate', 'orderdate_lastord', 'hotelid_lastord'
]].fillna(0).drop_duplicates(ucols)

df

df.sort_values('orderdate_lastord').index == df.sort_values('orderdate').index

if all(df.sort_values('orderdate_lastord') == df.sort_values('orderdate')):
    print(df.uid.unique()[0])

uc.loc[uc.orderid==2].shape, uc.loc[uc.orderid==3].shape

for i in uc.loc[uc.orderid==3].index:
    
    if check_time_error(train_df, i):
        print(i, 'success')
    else:
        print('error', i)

train_df[ucols].fillna(0).drop_duplicates([
    'user_confirmtime', 'user_avgadvanceddate', 'user_avgstar',
    'user_avggoldstar', 'user_avgrecommendlevel', 'user_avgroomnum', 'uid'
]).shape



uorder = train_df[['uid', 'orderid']].drop_duplicates()

ucount = uorder.groupby('uid').count()

ucount.loc[ucount.orderid>2]

ucount.describe()

train_df.groupby('uid').orderid.count().sort_values()

udf.drop_duplicates('uid').shape

udf.drop_duplicates().shape

np.abs(corr['orderlabel']).sort_values(ascending=False)

train_df.orderbehavior_3_ratio_1month.value_counts()

corr['orderlabel']['roomtag_3']

test_df.roomtag_2_lastord.value_counts()

train_df.roomtag_2_lastord.value_counts()

/home/zhanglun/.pyenv/versions/anaconda3-4.1.0/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':


## 物理属性分析

- hotelid 酒店 ((18543,), (34632,))
- basicroomid 物理房型((108611,), (192816,))
- roomid 子房型 ((811475, 151), (978298, 149))

## 酒店

训练集有3万个酒店，测试集也有三万，但是测试集总共订单也为3万，所以测试集基本上每个酒店只有一个订单


每个酒店平均接单1.64个，也就是说采样数据较为分散

每个房型 roomservice_1 保持不变
但是 roomservice_2-6 roomtag_1-4 会根据用户，根据时间变化（抽样结果线上，相同时间有可能会根据用户变化，相同用户会根据不同时间变化）


In [88]:
roomcols = train_df.columns[train_df.columns.str.extract('(room)').notnull()]

/home/zhanglun/.pyenv/versions/anaconda3-4.1.0/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':


In [21]:
test_df.orderid.value_counts().shape

(34632,)

In [68]:
train_df.basicroomid.value_counts().shape, test_df.basicroomid.value_counts().shape

((108611,), (192816,))

In [7]:
train_df.hotelid.value_counts().shape, test_df.hotelid.value_counts().shape

((18543,), (34632,))

In [70]:
train_df.drop_duplicates(['hotelid', 'roomid']).shape, test_df.drop_duplicates(['hotelid', 'roomid']).shape

((811475, 151), (978298, 149))

In [16]:
train_df.loc[train_df.orderlabel==1].groupby('hotelid').uid.count().describe()

count    18543.000000
mean         1.646120
std          1.469935
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max         33.000000
Name: uid, dtype: float64

In [17]:
train_df.loc[train_df.orderlabel==1].groupby('basicroomid').uid.count().describe()

count    23917.000000
mean         1.276247
std          0.867107
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max         30.000000
Name: uid, dtype: float64

In [67]:
train_df.loc[train_df.orderlabel==1].groupby(['hotelid', 'roomid']).uid.count().describe()

count    28695.000000
mean         1.063739
std          0.304609
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max         10.000000
Name: uid, dtype: float64

In [8]:
train_df.basicroomid.value_counts().shape, test_df.basicroomid.value_counts().shape

((108611,), (192816,))

In [10]:
train_df.roomid.value_counts().shape, test_df.roomid.value_counts().shape

((804370,), (974533,))

In [23]:
train_df['real_price'] = train_df['returnvalue'] + train_df['price_deduct']

In [72]:
for i in range(1, 9):
    print(train_df[['roomservice_%s'%i, 'hotelid', 'roomid']].drop_duplicates().shape)

(811475, 3)
(818623, 3)
(811951, 3)
(811544, 3)
(834782, 3)
(836001, 3)
(813669, 3)
(841460, 3)


In [74]:
for i in range(1, 5):
    print(train_df[['roomtag_%s'%i, 'hotelid', 'roomid']].drop_duplicates().shape)

(816476, 3)
(850356, 3)
(860581, 3)
(827531, 3)


In [110]:
ndf = train_df[['roomservice_2', 'hotelid', 'roomid']].drop_duplicates()

In [111]:
ncount = ndf.groupby(['hotelid', 'roomid']).count()

In [112]:
index = ncount.loc[ncount.roomservice_2>1].index

In [113]:
index.get_level_values(1)

Int64Index([   31043,    84213,    84217,   339011,   529611,   529613,
              529717,   529719,   529721,   529811,
            ...
            11434385, 11434405, 11434427, 11434495, 11434555, 11434575,
            11434605, 11436863, 11436883, 11436923],
           dtype='int64', name='roomid', length=7148)

In [122]:
d = train_df.orderdate[:1]

In [123]:
d.astype(np.int)

0    1497140524000000000
Name: orderdate, dtype: int64

In [127]:
for x in index.get_level_values(1)[:]:
    s = train_df.loc[train_df.roomid==x]['orderdate'].dt.day.std()
    if s == 0:
        print(train_df.loc[train_df.roomid==x][['orderdate', 'roomservice_2']])
        

                 orderdate  roomservice_2
295654 2017-06-08 06:56:23              1
991712 2017-06-08 14:16:37              0
                 orderdate  roomservice_2
231267 2017-06-11 18:55:16              1
726046 2017-06-11 08:00:18              0
                 orderdate  roomservice_2
231249 2017-06-11 18:55:16              1
726018 2017-06-11 08:00:18              0
                 orderdate  roomservice_2
231274 2017-06-11 18:55:16              1
726059 2017-06-11 08:00:18              0
                 orderdate  roomservice_2
70925  2017-06-11 12:05:29              0
231266 2017-06-11 18:55:16              1
726060 2017-06-11 08:00:18              0
                 orderdate  roomservice_2
231238 2017-06-11 18:55:16              1
726025 2017-06-11 08:00:18              0
                 orderdate  roomservice_2
521458 2017-06-08 13:38:48              1
678796 2017-06-08 02:41:47              0
                 orderdate  roomservice_2
600456 2017-06-05 22:57:35        

In [108]:
train_df.loc[train_df.roomid==31043]['orderdate'].dt.day.std()

0.0

In [106]:
train_df.loc[train_df.roomid==5534421]

,orderid,uid,orderdate,hotelid,basicroomid,roomid,orderlabel,star,rank,returnvalue,...,roomservice_6_lastord,roomservice_8_lastord,roomtag_2_lastord,roomtag_3_lastord,roomtag_4_lastord,roomtag_5_lastord,star_lastord,hotel_minprice_lastord,basic_minprice_lastord,real_price
385956,63264,123284,2017-06-09 08:53:53,649,4679,5534421,0,11,21,200.0,...,0.0,5.0,NaN,458.0,0.0,0.0,11.0,1169.0,1169.0,1600.0
636700,45932,123284,2017-06-08 06:53:48,649,4679,5534421,0,11,21,206.0,...,0.0,5.0,313.0,444.0,0.0,0.0,11.0,1175.0,1175.0,1600.0


In [103]:
for i in index.get_level_values(1):
    s = train_df.loc[train_df.roomid==i]['uid'].unique().shape[0]
    if s == 1:
        print('gg', i)

gg 5534421
gg 5536819
gg 5537915
gg 5539219
gg 7314127
gg 858417
gg 13987021
gg 13990219
gg 13990221
gg 2299597
gg 22995415
gg 3083783
gg 30833723
gg 30834227
gg 30837623
gg 3364495
gg 3364673
gg 3364849
gg 33646011
gg 3983533
gg 3983607
gg 71238311
gg 9421727
gg 11004213
gg 11004215
gg 11004223
gg 11004243
gg 11004263


### 酒店星级

子房型rank 由物理房型和酒店子房型确定

酒店star 1-11(难道不是按5星级来划分？）， 测试的星级要比训练的大



In [49]:
hrank = test_df.drop_duplicates(['basicroomid', 'roomid', 'rank'])

In [50]:
hrank.groupby(['basicroomid', 'roomid']).uid.count().describe()

count    978298.0
mean          1.0
std           0.0
min           1.0
25%           1.0
50%           1.0
75%           1.0
max           1.0
Name: uid, dtype: float64

In [55]:
train_df.star.describe()

count    1.094826e+06
mean     8.557309e+00
std      2.587717e+00
min      1.000000e+00
25%      7.000000e+00
50%      9.000000e+00
75%      1.100000e+01
max      1.100000e+01
Name: star, dtype: float64

In [44]:
train_df[['orderdate', 'star']].set_index('orderdate').resample('D').mean()

,star
orderdate,
2017-06-05,8.616157
2017-06-06,8.609248
2017-06-07,8.660252
2017-06-08,8.499137
2017-06-09,8.517103
2017-06-10,8.495958
2017-06-11,8.509195


In [51]:
test_df['star'].describe()

count    978298.000000
mean          6.823575
std           2.865185
min           1.000000
25%           5.000000
50%           7.000000
75%           9.000000
max          11.000000
Name: star, dtype: float64

In [148]:
test_df.loc[test_df['rank']<65]['rank'].describe()

count    978009.000000
mean          8.903394
std           5.872304
min           3.000000
25%           5.000000
50%           7.000000
75%          11.000000
max          63.000000
Name: rank, dtype: float64

In [ ]:
train_df.loc[train_df.orderdate==1]

In [13]:
train_df['rank'].describe()

count    1.094826e+06
mean     1.106601e+01
std      1.198979e+01
min      3.000000e+00
25%      5.000000e+00
50%      9.000000e+00
75%      1.300000e+01
max      5.450000e+02
Name: rank, dtype: float64

In [151]:
corr = train_df[c].sample(100000).corr()

In [156]:
abs(corr.orderlabel).sort_values(ascending=False)

orderlabel                          1.000000
roomtag_1                           0.406815
room_30days_ordnumratio             0.220140
basic_week_ordernum_ratio           0.151447
basic_30days_ordnumratio            0.151420
basic_recent3_ordernum_ratio        0.144302
basic_comment_ratio                 0.126720
rank                                0.082593
basic_30days_realratio              0.065626
star                                0.062201
roomservice_8                       0.060242
roomid                              0.054190
roomservice_3                       0.050297
basicroomid                         0.047506
hotelid                             0.047016
room_30days_realratio               0.041762
user_avgprice_star                  0.038415
user_avgstar                        0.035385
orderbehavior_9                     0.035058
user_avgprice_1month                0.032985
user_maxprice_1month                0.032165
star_lastord                        0.031966
real_price

## 测试和训练用户和酒店交叉的部分 

In [88]:
def test_cross(f):
    truid = train_df[[f]].drop_duplicates()
    print('train', truid.shape)

    teuid = test_df[[f]].drop_duplicates()
    print('test', teuid.shape)
    teuid['is_test'] = 1
    teuid = teuid.set_index(f)

    df = truid.join(teuid, on=f, how='outer')
    df.info(null_counts=True, verbose=True)
    print(teuid.shape[0] + truid.shape[0] - df.shape[0], 'cross')

###  用户

训练测试各3万用户，交叉817个

In [89]:
test_cross('uid')

train (29960, 1)
test (34632, 1)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 63765 entries, 0 to 1094738
Data columns (total 2 columns):
uid        63765 non-null int64
is_test    34632 non-null float64
dtypes: float64(1), int64(1)
memory usage: 1.5 MB
827 cross


### 酒店

测试3.5万， 训练1.9万，交叉1.1万

In [87]:
test_cross('hotelid')

train (18543, 1)
test (34632, 1)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 42208 entries, 0 to 1094495
Data columns (total 2 columns):
hotelid    42208 non-null int64
is_test    34632 non-null float64
dtypes: float64(1), int64(1)
memory usage: 989.2 KB
10967 cross


In [90]:
test_cross('basicroomid')

train (108611, 1)
test (192816, 1)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 239130 entries, 0 to 1094507
Data columns (total 2 columns):
basicroomid    239130 non-null int64
is_test        192816 non-null float64
dtypes: float64(1), int64(1)
memory usage: 5.5 MB
62297 cross


In [91]:
test_cross('roomid')

train (804370, 1)
test (974533, 1)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1567740 entries, 0 to 1094815
Data columns (total 2 columns):
roomid     1567740 non-null int64
is_test    974533 non-null float64
dtypes: float64(1), int64(1)
memory usage: 35.9 MB
211163 cross


### 子房型价格

In [92]:
train_df['real_price'] = train_df.price_deduct + train_df.returnvalue

In [95]:
train_df.loc[train_df.orderlabel==1].groupby(train_df.orderdate.dt.day).real_price.mean()

orderdate
5     1627.838501
6     1632.695679
7     1693.120850
8     1633.515137
9     1612.257690
10    1625.537842
11    1601.703003
Name: real_price, dtype: float32

In [96]:
train_df.loc[train_df.orderlabel!=5].groupby(train_df.orderdate.dt.day).real_price.mean()

orderdate
5     3128.936279
6     3184.863770
7     3250.051514
8     3100.873291
9     3088.576660
10    3138.731689
11    3160.123535
Name: real_price, dtype: float32

In [99]:
cat = train_df.orderdate.astype('category')

## 酒店属性对应关系


- hotelid 酒店 ((18543,), (34632,))
- basicroomid 物理房型((108611,), (192816,))
- roomid 子房型(hotelid+roomid) ((811475, 151), (978298, 149))

- star  --- hotelid

- rank --- roomid

- returnvalue  --- orderid + uid + roomid

- price_deduct --- orderid + uid + roomid

- basic_minarea  --- hotelid + roomid

- basic_maxarea --- hotelid + roomid

- roomservice_[1-8] --- hotelid + roomid

差值

-  roomservice_1 train 0 test 0
-  roomservice_2 train 7148 test 0
-  roomservice_3 train 476 test 0
-  roomservice_4 train 69 test 0
-  roomservice_5 train 23307 test 0
-  roomservice_6 train 24526 test 0
-  roomservice_7 train 2194 test 0
-  roomservice_8 train 29985 test 0

差值
- roomtag_[1-4] --- hotelid + roomid

-  roomtag_1 train 5001 test 0
-  roomtag_2 train 38881 test 0
-  roomtag_3 train 49106 test 0
-  roomtag_4 train 16056 test 0

归纳属性基于天来统计

-  basic_week_ordernum_ratio train 43347 test 0
-  basic_recent3_ordernum_ratio train 40266 test 0
-  basic_comment_ratio train 40910 test 0
-  basic_30days_ordnumratio train 45894 test 0
-  basic_30days_realratio train 25776 test 0

-  room_30days_ordnumratio train 130355 test 0
-  room_30days_realratio train 65861 test 0


In [86]:
uocount = train_df.drop_duplicates(['uid', 'orderid']).groupby('uid').orderid.count()

In [87]:
buy_t = uocount.loc[uocount>1]

In [112]:
# 测试去除购买两次以上的用户对房型属性的影响,结果还是一样,可能数据错误或者房型对用户或者时间改变了
# ntrain = train_df.join(buy_t, on='uid', rsuffix='buymany')

# train_df = ntrain.loc[ntrain.orderidbuymany.isnull()]

# train_df.orderdate.max()

# train_df = train_df.set_index('orderdate').loc['2017-06-11'].reset_index()

In [126]:
for i in range(1, 9):
    t = 'roomservice_%d' % i
    s1 =train_df.drop_duplicates([t, 'roomid', 'hotelid']).shape[0]
    s2 = train_df.drop_duplicates([ 'roomid', 'hotelid']).shape[0]
    s3 =test_df.drop_duplicates([t, 'roomid', 'hotelid']).shape[0]
    s4 = test_df.drop_duplicates([ 'roomid', 'hotelid']).shape[0]
    print('- ', t, 'train', s1-s2, 'test', s3 -s4)


-  roomservice_1 train 0 test 0
-  roomservice_2 train 7148 test 0
-  roomservice_3 train 476 test 0
-  roomservice_4 train 69 test 0
-  roomservice_5 train 23307 test 0
-  roomservice_6 train 24526 test 0
-  roomservice_7 train 2194 test 0
-  roomservice_8 train 29985 test 0


In [150]:
for i in range(1, 5):
    t = 'roomtag_%d' % i
    s1 =train_df.drop_duplicates([t, 'roomid', 'hotelid']).shape[0]
    s2 = train_df.drop_duplicates([ 'roomid', 'hotelid']).shape[0]
    s3 =test_df.drop_duplicates([t, 'roomid', 'hotelid']).shape[0]
    s4 = test_df.drop_duplicates([ 'roomid', 'hotelid']).shape[0]
    print('- ', t, 'train', s1-s2, 'test', s3 -s4)


-  roomtag_1 train 148 test 0
-  roomtag_2 train 1624 test 0
-  roomtag_3 train 737 test 0
-  roomtag_4 train 542 test 0


In [130]:
basic_r = [x for x in train_df.columns if x.startswith('basic') and x.endswith('ratio')]

In [131]:
room_r = [x for x in train_df.columns if x.startswith('room') and x.endswith('ratio')]

In [137]:
# # 测试去除购买两次以上的用户对房型属性的影响,结果还是一样,可能数据错误或者房型对用户或者时间改变了
# old_t = train_df

# ntrain = train_df.join(buy_t, on='uid', rsuffix='buymany')

# train_df = ntrain.loc[ntrain.orderidbuymany.isnull()]


# train_df.orderdate.max()

# train_df = train_df.set_index('orderdate').loc['2017-06-11'].reset_index()

In [141]:
# train_df = old_t

In [ ]:
train_df.basicroomid

In [149]:
for t in basic_r:
    s1 =train_df.drop_duplicates([t, 'basicroomid']).shape[0]
    s2 = train_df.drop_duplicates([ 'basicroomid']).shape[0]
    s3 =test_df.drop_duplicates([t, 'basicroomid']).shape[0]
    s4 = test_df.drop_duplicates([ 'basicroomid']).shape[0]
    print('- ', t, 'train', s1-s2, 'test', s3 -s4)


-  basic_week_ordernum_ratio train 0 test 0
-  basic_recent3_ordernum_ratio train 0 test 0
-  basic_comment_ratio train 0 test 0
-  basic_30days_ordnumratio train 0 test 0
-  basic_30days_realratio train 0 test 0


In [148]:
for t in room_r:
    s1 =train_df.drop_duplicates([t, 'roomid', 'hotelid']).shape[0]
    s2 = train_df.drop_duplicates([ 'roomid', 'hotelid']).shape[0]
    s3 =test_df.drop_duplicates([t, 'roomid', 'hotelid']).shape[0]
    s4 = test_df.drop_duplicates([ 'roomid', 'hotelid']).shape[0]
    print('- ', t, 'train', s1-s2, 'test', s3 -s4)


-  room_30days_ordnumratio train 0 test 0
-  room_30days_realratio train 0 test 0


In [60]:
train_df.roomtag_5_lastord.value_counts()

0.0    995202
1.0      1939
Name: roomtag_5_lastord, dtype: int64

In [56]:
test_df.drop_duplicates(['roomservice_2', 'roomid', 'hotelid']).shape

(978298, 149)

In [57]:
test_df.drop_duplicates([ 'roomid', 'hotelid']).shape

(978298, 149)

## 用户属性对应关系

用户 ((29960, train), (34632, test))


去除用户后,无差异

- user_confirmtime train 216 test 0
- user_avgadvanceddate train 183 test 0
- user_avgstar train 166 test 0
- user_avggoldstar train 214 test 0
- user_avgrecommendlevel train 195 test 0
- user_avgroomnum train 25 test 0


-  ordertype_1_ratio train 34 test 0
-  ordertype_2_ratio train 242 test 0
-  ordertype_3_ratio train 18 test 0
-  ordertype_4_ratio train 170 test 0
-  ordertype_5_ratio train 50 test 0
-  ordertype_6_ratio train 372 test 0
-  ordertype_7_ratio train 382 test 0
-  ordertype_8_ratio train 193 test 0
-  ordertype_9_ratio train 200 test 0
-  ordertype_10_ratio train 356 test 0
-  ordertype_11_ratio train 223 test 0


-  user_confirmtime train 216 test 0
-  user_avgadvanceddate train 183 test 0
-  user_avgstar train 166 test 0
-  user_avggoldstar train 214 test 0
-  user_avgrecommendlevel train 195 test 0
-  user_avgroomnum train 25 test 0
-  user_avgdealpriceholiday train 34 test 0
-  user_avgdealpriceworkday train 196 test 0
-  user_avgdealprice train 201 test 0
-  user_avgpromotion train 87 test 0
-  user_avgprice_star train 396 test 0
-  user_ordernum train 391 test 0
-  user_activation train 325 test 0
-  user_avgprice train 382 test 0
-  user_maxprice train 36 test 0
-  user_minprice train 41 test 0
-  user_stdprice train 359 test 0
-  user_cvprice train 353 test 0
-  user_citynum train 144 test 0
-  user_avgroomarea train 4 test 0
-  user_roomservice_4_0ratio train 78 test 0
-  user_roomservice_4_2ratio train 342 test 0
-  user_roomservice_4_3ratio train 332 test 0
-  user_roomservice_4_4ratio train 207 test 0
-  user_roomservice_4_1ratio train 23 test 0
-  user_roomservice_4_5ratio train 101 test 0
-  user_roomservice_3_123ratio train 299 test 0
-  user_roomservice_6_2ratio train 309 test 0
-  user_roomservice_6_1ratio train 225 test 0
-  user_roomservice_6_0ratio train 327 test 0
-  user_roomservice_5_1ratio train 330 test 0
-  user_roomservice_7_0ratio train 320 test 0
-  user_roomservice_2_1ratio train 246 test 0
-  user_roomservice_8_1ratio train 339 test 0
-  user_rank_ratio train 362 test 0
-  user_roomservice_8_345ratio train 293 test 0
-  user_ordnum_1week train 367 test 0
-  user_avgprice_1week train 383 test 0
-  user_medprice_1week train 339 test 0
-  user_minprice_1week train 252 test 0
-  user_maxprice_1week train 275 test 0
-  user_roomservice_3_123ratio_1week train 260 test 0
-  user_roomservice_7_1ratio_1week train 260 test 0
-  user_roomservice_7_0ratio_1week train 260 test 0
-  user_roomservice_4_5ratio_1week train 168 test 0
-  user_roomservice_4_4ratio_1week train 217 test 0
-  user_roomservice_4_2ratio_1week train 272 test 0
-  user_roomservice_4_3ratio_1week train 263 test 0
-  user_roomservice_4_0ratio_1week train 174 test 0
-  user_ordnum_1month train 379 test 0
-  user_avgprice_1month train 385 test 0
-  user_medprice_1month train 318 test 0
-  user_minprice_1month train 172 test 0
-  user_maxprice_1month train 194 test 0
-  user_roomservice_3_123ratio_1month train 253 test 0
-  user_roomservice_7_1ratio_1month train 267 test 0
-  user_roomservice_7_0ratio_1month train 267 test 0
-  user_roomservice_4_5ratio_1month train 92 test 0
-  user_roomservice_4_4ratio_1month train 185 test 0
-  user_roomservice_4_2ratio_1month train 276 test 0
-  user_roomservice_4_3ratio_1month train 256 test 0
-  user_roomservice_4_0ratio_1month train 105 test 0
-  user_ordnum_3month train 375 test 0
-  user_avgprice_3month train 389 test 0
-  user_medprice_3month train 299 test 0
-  user_minprice_3month train 106 test 0
-  user_maxprice_3month train 110 test 0
-  user_roomservice_3_123ratio_3month train 274 test 0
-  user_roomservice_7_1ratio_3month train 286 test 0
-  user_roomservice_7_0ratio_3month train 286 test 0
-  user_roomservice_4_5ratio_3month train 56 test 0
-  user_roomservice_4_4ratio_3month train 188 test 0
-  user_roomservice_4_2ratio_3month train 309 test 0
-  user_roomservice_4_3ratio_3month train 287 test 0
-  user_roomservice_4_0ratio_3month train 75 test 0

In [61]:
train_df.uid.unique().shape, test_df.uid.unique().shape

((29960,), (34632,))

In [105]:
uocount = train_df.drop_duplicates(['uid', 'orderid']).groupby('uid').orderid.count()

In [108]:
uocount.loc[uocount>1].shape

(538,)

In [106]:
# buy_t = uocount.loc[uocount>1]

# ntrain = train_df.join(buy_t, on='uid', rsuffix='buymany')

# train_df = ntrain.loc[ntrain.orderidbuymany.isnull()]


In [147]:
for i in ['confirmtime', 'avgadvanceddate', 'avgstar', 'avggoldstar', 'avgrecommendlevel', 'avgroomnum']:
    t = 'user_%s' % i
    s1 =train_df.drop_duplicates([t, 'uid']).shape[0]
    s2 = train_df.drop_duplicates([ 'uid']).shape[0]
    s3 =test_df.drop_duplicates([t, 'uid']).shape[0]
    s4 = test_df.drop_duplicates([ 'uid']).shape[0]
    print('-', t, 'train', s1-s2, 'test', s3 -s4)


- user_confirmtime train 0 test 0
- user_avgadvanceddate train 0 test 0
- user_avgstar train 0 test 0
- user_avggoldstar train 0 test 0
- user_avgrecommendlevel train 0 test 0
- user_avgroomnum train 0 test 0


In [113]:
[x for x in train_df.columns if x.endswith('ratio') and x.startswith('ordertype')]

['ordertype_1_ratio',
 'ordertype_2_ratio',
 'ordertype_3_ratio',
 'ordertype_4_ratio',
 'ordertype_5_ratio',
 'ordertype_6_ratio',
 'ordertype_7_ratio',
 'ordertype_8_ratio',
 'ordertype_9_ratio',
 'ordertype_10_ratio',
 'ordertype_11_ratio']

In [120]:
# buy_t = uocount.loc[uocount>1]

# ntrain = train_df.join(buy_t, on='uid', rsuffix='buymany')

# train_df = ntrain.loc[ntrain.orderidbuymany.isnull()]


In [123]:
# train_df = pd.read_pickle('../dataset/train.pkl')

In [146]:
for t in [x for x in train_df.columns if x.endswith('ratio') and x.startswith('ordertype')]:
    s1 =train_df.drop_duplicates([t, 'uid']).shape[0]
    s2 = train_df.drop_duplicates([ 'uid']).shape[0]
    s3 =test_df.drop_duplicates([t, 'uid']).shape[0]
    s4 = test_df.drop_duplicates([ 'uid']).shape[0]
    print('- ', t, 'train', s1-s2, 'test', s3 -s4)


-  ordertype_1_ratio train 0 test 0
-  ordertype_2_ratio train 0 test 0
-  ordertype_3_ratio train 0 test 0
-  ordertype_4_ratio train 0 test 0
-  ordertype_5_ratio train 0 test 0
-  ordertype_6_ratio train 0 test 0
-  ordertype_7_ratio train 0 test 0
-  ordertype_8_ratio train 0 test 0
-  ordertype_9_ratio train 0 test 0
-  ordertype_10_ratio train 0 test 0
-  ordertype_11_ratio train 0 test 0


In [145]:
for t in [x for x in train_df.columns if not x.endswith('id') and x.startswith('user')]:
    s1 =train_df.drop_duplicates([t, 'uid']).shape[0]
    s2 = train_df.drop_duplicates([ 'uid']).shape[0]
    s3 =test_df.drop_duplicates([t, 'uid']).shape[0]
    s4 = test_df.drop_duplicates([ 'uid']).shape[0]
    print('- ', t, 'train', s1-s2, 'test', s3 -s4)


-  user_confirmtime train 0 test 0
-  user_avgadvanceddate train 0 test 0
-  user_avgstar train 0 test 0
-  user_avggoldstar train 0 test 0
-  user_avgrecommendlevel train 0 test 0
-  user_avgroomnum train 0 test 0
-  user_avgdealpriceholiday train 0 test 0
-  user_avgdealpriceworkday train 0 test 0
-  user_avgdealprice train 0 test 0
-  user_avgpromotion train 0 test 0
-  user_avgprice_star train 6 test 0
-  user_ordernum train 0 test 0
-  user_activation train 0 test 0
-  user_avgprice train 0 test 0
-  user_maxprice train 0 test 0
-  user_minprice train 0 test 0
-  user_stdprice train 0 test 0
-  user_cvprice train 0 test 0
-  user_citynum train 0 test 0
-  user_avgroomarea train 0 test 0
-  user_roomservice_4_0ratio train 0 test 0
-  user_roomservice_4_2ratio train 0 test 0
-  user_roomservice_4_3ratio train 0 test 0
-  user_roomservice_4_4ratio train 0 test 0
-  user_roomservice_4_1ratio train 0 test 0
-  user_roomservice_4_5ratio train 0 test 0
-  user_roomservice_3_123ratio trai

In [63]:
train_df.drop_duplicates(['user_confirmtime', 'uid']).shape

(30176, 150)

In [64]:
test_df.drop_duplicates(['user_confirmtime', 'uid']).shape

(34632, 149)

In [153]:
for t in [x for x in train_df.columns if  x.endswith('lastord') and x != 'orderlabel']:
    s1 =train_df.drop_duplicates([t, 'uid']).shape[0]
    s2 = train_df.drop_duplicates([ 'uid']).shape[0]
    s3 =test_df.drop_duplicates([t, 'uid']).shape[0]
    s4 = test_df.drop_duplicates([ 'uid']).shape[0]
    print('- ', t, 'train', s1-s2, 'test', s3 -s4)


-  orderdate_lastord train 0 test 0
-  hotelid_lastord train 0 test 0
-  roomid_lastord train 0 test 0
-  basicroomid_lastord train 0 test 0
-  rank_lastord train 0 test 0
-  return_lastord train 0 test 0
-  price_last_lastord train 0 test 0
-  roomservice_2_lastord train 0 test 0
-  roomservice_3_lastord train 0 test 0
-  roomservice_4_lastord train 0 test 0
-  roomservice_5_lastord train 0 test 0
-  roomservice_6_lastord train 0 test 0
-  roomservice_8_lastord train 0 test 0
-  roomtag_2_lastord train 0 test 0
-  roomtag_3_lastord train 0 test 0
-  roomtag_4_lastord train 0 test 0
-  roomtag_5_lastord train 0 test 0
-  star_lastord train 0 test 0
-  hotel_minprice_lastord train 0 test 0
-  basic_minprice_lastord train 0 test 0
